# 0 导入必要的包

In [69]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import ucTools
import funclib
sys.path.append("../../")
import train
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. 读取数据

In [2]:
# 1515 数据
sprot1515 = pd.read_csv('./data/ecoli1515/1515testset_with_unirep.tsv',  sep='\t')

In [3]:
sprot1515

,Unnamed: 0,id,ec_number,seq,isemzyme,isMultiFunctional,blattner_id,blattner_ec_number,0,1,...,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899
0,0,P42641,3.6.5.-,MKFVDEASILVVAGDGGNGCVSFRREKYIPKGGPDGGDGGDGGDVW...,True,False,b3183,NaN,0.003056,-0.087184,...,0.000332,-0.065542,0.117107,-0.236777,-0.622398,0.012113,0.010506,0.058984,0.061533,0.968014
1,1,P0AE22,3.1.3.2,MRKITQAISAVCLLFALNSSAVALASSPSPLNPGTNVARLAEQAPI...,True,False,b4055,3.1.3.2,0.001026,0.618393,...,0.030307,-0.244636,-0.038585,0.151263,-0.020389,0.095321,-0.031043,-0.881730,-0.196211,0.054455
2,2,P21499,3.1.13.1,MSQDPFQEREAEKYANPIPSREFILEHLTKREKPASRDELAVELHI...,True,False,b4179,3.1.-.-,0.073632,-0.003258,...,0.002123,0.060351,-0.008826,0.006141,-0.129352,-0.075010,0.023712,-0.002246,0.893310,0.140025
3,3,P39286,3.6.1.-,MSKNKLSKGQQRRVNANHQRRLKTSKEKPDYDDNLFGEPDEGIVIS...,True,False,b4161,3.6.1.-,0.000775,-0.008305,...,0.066885,0.020991,0.073577,-0.034962,-0.036755,0.053803,-0.017355,0.100456,-0.202344,0.351100
4,4,P56100,7.1.1.7,MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI,True,False,b4515,1.10.3.10,0.001811,-0.050181,...,0.042016,-0.028950,0.305668,-0.105587,-0.113769,0.680413,0.138797,-0.268528,-0.076093,-0.038212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,4745,P36837,NaN,MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...,False,False,b3496,NaN,0.000070,-0.021424,...,0.005911,-0.016855,0.377301,-0.500727,-0.247861,0.752678,-0.005501,0.396191,-0.215832,-0.062812
4746,4746,P77304,NaN,MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...,False,False,b1634,NaN,0.008628,-0.065382,...,-0.001197,-0.101824,0.013007,-0.304450,-0.295525,-0.124492,0.010846,0.032853,-0.155702,-0.183535
4747,4747,P27248,2.1.2.10,MAQQTPLYEQHTLCGARMVDFHGWMMPLHYGSQIDEHHAVRTDAGM...,True,False,b2905,2.1.2.10,0.006723,0.227349,...,-0.124573,-0.032099,-0.170996,0.104425,-0.163041,0.183720,-0.038814,0.036993,-0.044558,0.010447
4748,4748,P0AC88,4.2.1.47,MSKVALITGVTGQDGSYLAEFLLEKGYEVHGIKRRASSFNTERVDH...,True,False,b2053,4.2.1.47,0.000665,-0.029179,...,0.145032,0.867074,-0.263008,0.009123,0.036783,-0.073805,0.439632,0.014501,-0.205913,-0.005663


In [4]:
funclib.table2fasta(sprot1515, './data/sprot1515.fasta')

Write finished


## 2. 序列比对

In [5]:
train = pd.read_hdf('./data/train.h5',key='data')
test = pd.read_hdf('./data/test.h5',key='data')
head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
head = head + ['ec_label','ec_appears']
train.columns = head
test.columns = head

In [ ]:
train = pd.read_hdf('./data/train.h5',key='data')

In [58]:
file_enzyme_train_x = '../../data/preprocess/enzyme_train_x.feather'
file_enzyme_train_y = '../../data/preprocess/enzyme_train_y.feather'
DATE_TRAIN ='2018-01-01'
uctools =  ucTools.ucTools('172.16.25.20')
cnx_ecnumber = uctools.db_conn()

In [59]:
enzyme_X, enzyme_Y = train.get_enzyme_train_set( train_date=DATE_TRAIN, 
                            eclablefile='../../data/dict_ec_label.npy', 
                            featurefile='../../data/sprot_unirep.feather', 
                            cnx=cnx_ecnumber,
                            trainX=file_enzyme_train_x,
                            trainY=file_enzyme_train_y
                          )

In [60]:
file_isenzyme_model = './model/isenzyme.model'
train.train_isenzyme(enzyme_X, enzyme_Y.enzyme_label, model_file=file_isenzyme_model)

XGBoost模型训练完成


In [70]:
eclablefile='../../data/dict_ec_label.npy'
featurefile='../../data/sprot_unirep.feather'

dict_ec_label, dict_label_ec = train.load_ec_lable_dict(eclablefile)
train_data = train.load_train_file(split_date= DATE_TRAIN, cnx_ecnumber=cnx_ecnumber, type='ec')

#添加训练标签
train_data_labeled  = train.add_ec_label(train_data, dict_ec_label)

unirep_data = train.load_unirep(featurefile)

res_data = train_data_labeled.merge(unirep_data, on='id', how='left')

AttributeError: module 'train' has no attribute 'train_data_labeled'

In [71]:
res_data = train_data_labeled.merge(unirep_data, on='id', how='left')

In [72]:
res_data

,id,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,seq,ec_label,enzyme_label,f1,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,Q9KSU0,True,False,1,6.1.1.12,4,MRSHYCGHLNKSLVGQTVELCGWVNRRRDLGGLIFIDMRDREGIVQ...,5108,1,0.001720,...,0.008382,0.222804,-0.003487,0.047094,-0.160011,0.263211,-0.002514,0.059199,-0.169309,-0.011958
1,B1JUC0,True,False,1,6.3.5.2,4,MHDKILILDFGSQVTQLIARRVREAHVYCEIHPNDVSDDFVREFAP...,5275,1,0.000716,...,0.071157,0.019109,0.048535,0.759038,-0.060638,0.085412,-0.009784,0.000936,-0.069721,0.095841
2,Q0BE45,True,False,1,6.3.5.2,4,MHDKILILDFGSQVTQLIARRVREAHVYCEIHPNDVSDEFVREFAP...,5275,1,0.000741,...,0.079030,0.016518,0.049998,0.725356,-0.058271,0.092345,-0.009605,0.001112,-0.071002,0.088221
3,B1JK22,True,False,1,5.4.3.8,4,MSKSENLYAQAQQLIPGGVNSPVRAFTGVGGIPLFIERADGAYLFD...,4994,1,0.000043,...,0.067364,0.001963,0.209237,-0.453424,-0.067034,0.020088,0.041261,0.074060,-0.008788,0.768053
4,B1ZGA6,True,False,1,5.4.99.12,4,MPRYKLVIEYDGGPFCGWQRQADDPTVQAAIETAVTRFSGETARLT...,5005,1,0.009229,...,0.030803,-0.023426,-0.043002,0.204930,0.013927,0.057860,-0.017096,-0.901541,-0.154073,0.294186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243972,Q37706,True,False,1,7.1.1.9,4,MSQWFQLGLQNGNSPLMEQLIFFHDHALLVVILITSLVGFFLAALF...,5311,1,0.000192,...,0.036817,-0.130493,-0.301297,0.008477,-0.107571,0.153052,0.437314,-0.319994,-0.364219,0.008638
243973,A5VU71,True,False,1,6.3.5.2,4,MSTTAYPDTILIIDFGSQVTQLIARRVREANVYCEIVPFQSADEAF...,5275,1,0.001035,...,0.156979,-0.028586,-0.012056,0.554148,-0.070553,0.037575,-0.004441,0.002646,-0.072896,0.262390
243974,C0ZCX0,True,False,1,3.4.19.3,4,MKTILVTGFDPFGGEIVNPAWESVKELGKIESDLYKVELRQIPTVF...,3672,1,0.002699,...,0.101485,0.293245,-0.368090,0.298136,-0.109222,0.013300,0.483309,0.041904,-0.048383,-0.017721
243975,B7MAE6,True,False,1,6.1.1.5,4,MSDYKSTLNLPETGFPMRGDLAKREPGMLARWTDDDLYGIIRAAKK...,5124,1,0.000406,...,-0.001378,0.143041,-0.037172,0.052415,-0.111394,0.344566,-0.028339,-0.026787,-0.112824,0.135730


In [77]:
res_data.iloc[:,9:1908]

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f1890,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899
0,0.001720,-0.035647,0.189430,-0.003932,0.466307,-0.111438,0.039783,-0.002550,-0.001469,0.138039,...,0.148958,0.008382,0.222804,-0.003487,0.047094,-0.160011,0.263211,-0.002514,0.059199,-0.169309
1,0.000716,0.711519,0.197860,-0.028751,-0.537455,0.096588,-0.043127,0.070035,-0.021396,0.012285,...,0.229026,0.071157,0.019109,0.048535,0.759038,-0.060638,0.085412,-0.009784,0.000936,-0.069721
2,0.000741,0.713632,0.193882,-0.028563,-0.525715,0.099166,-0.032833,0.075919,-0.020634,0.011567,...,0.227315,0.079030,0.016518,0.049998,0.725356,-0.058271,0.092345,-0.009605,0.001112,-0.071002
3,0.000043,-0.252796,-0.111056,-0.957261,-0.084564,-0.327607,-0.029824,-0.011598,-0.004002,0.000734,...,0.536304,0.067364,0.001963,0.209237,-0.453424,-0.067034,0.020088,0.041261,0.074060,-0.008788
4,0.009229,0.575313,0.396739,-0.001204,-0.687519,0.060662,0.033489,-0.011678,-0.005659,-0.026520,...,0.079986,0.030803,-0.023426,-0.043002,0.204930,0.013927,0.057860,-0.017096,-0.901541,-0.154073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243972,0.000192,0.000267,0.017273,-0.009716,0.015150,0.006729,0.043155,-0.122820,-0.000990,0.111653,...,0.040652,0.036817,-0.130493,-0.301297,0.008477,-0.107571,0.153052,0.437314,-0.319994,-0.364219
243973,0.001035,0.723965,0.129614,-0.031151,-0.685001,0.041716,-0.031471,0.134439,-0.016206,0.003443,...,-0.021219,0.156979,-0.028586,-0.012056,0.554148,-0.070553,0.037575,-0.004441,0.002646,-0.072896
243974,0.002699,0.192036,0.055722,-0.017692,-0.638779,0.121626,0.030357,-0.071625,-0.006182,-0.051465,...,0.029733,0.101485,0.293245,-0.368090,0.298136,-0.109222,0.013300,0.483309,0.041904,-0.048383
243975,0.000406,0.096108,0.020882,-0.020346,-0.030447,0.056343,0.097987,-0.005727,-0.000381,-0.024931,...,0.073738,-0.001378,0.143041,-0.037172,0.052415,-0.111394,0.344566,-0.028339,-0.026787,-0.112824
